## Fetch Weather Data: From Stations Around the Wind Farms
### *This notebook downloads wind data from the NOAA API from the list of stations found in 02 - Fetch Weather Data.ipynb*

In [2]:
import requests
import json
import os
import pandas as pd
import numpy as np
from pyproj import Proj
from shapely.geometry import shape
import pickle
#headers = {"token": "xVEIkLnfHyheHhvoheZSxesUerlyrxGN"}
headers = {"token": "bgBKBpxaFICMeDMgPFYAMGyaDrBJMPwB"}

#### Stations
* Stations are where the data comes from (for most datasets) and can be considered the smallest granual of location data. 
* If the desired station is known, all of its data can quickly be viewed.

In [3]:
# read all project stations fetched from the NOAA API
all_project_stations = pd.read_csv('./data/all_project_stations.csv')
all_project_stations.head()

,datacoverage,elevation,elevationUnit,id,latitude,longitude,maxdate,mindate,name,square_diagonal,query_count,query_long,query_lat,dist_from_center,p_name
0,0.9995,304.8,METERS,GHCND_USW00013969,36.73667,-97.10194,2018-10-16,1948-01-01,"PONCA CITY MUNICIPAL AIRPORT, OK US",2.139864,14,-97.306297,36.572365,0.262218,Thunder Ranch
1,0.9872,299.9,METERS,GHCND_USW00003965,36.16240,-97.08940,2018-10-16,1996-11-01,"STILLWATER REGIONAL AIRPORT, OK US",2.139864,14,-97.306297,36.572365,0.463805,Thunder Ranch
2,0.9675,272.8,METERS,GHCND_USC00348501,36.11750,-97.09500,2018-10-17,1893-01-01,"STILLWATER 2 W, OK US",2.139864,14,-97.306297,36.572365,0.501546,Thunder Ranch
3,0.8607,1309.4,METERS,GHCND_USW00024030,41.09930,-102.98630,2018-10-16,1948-01-01,"SIDNEY MUNICIPAL AIRPORT, NE US",1.783220,13,-103.322228,40.961955,0.362920,Peetz Table
4,0.9565,1316.7,METERS,GHCND_USC00257830,41.22940,-103.02140,2018-09-30,1908-12-01,"SIDNEY 6 NNW, NE US",1.783220,13,-103.322228,40.961955,0.402522,Peetz Table


#### Get the dataypes for wind which we will be downloading the wind data

In [20]:
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/datatypes?datacategoryid=WIND&limit=56"
response = requests.get(url=url, headers=headers)
noaa_wind_data_types_json = response.json()
noaa_wind_data_types_df = pd.DataFrame(noaa_wind_data_types_json['results'])
wind_data_types = noaa_wind_data_types_df.id

In [5]:
len(wind_data_types)

27

In [6]:
#parameters for weather query
datasetid='GHCND'
datatypeid="AWND"
stationid='GHCND:USW00013969'
units='standard'
startdate='2017-01-01'
enddate='2017-12-31'
limit = 365

In [18]:
def get_wind_data_daily_summary(stationid, datatypeid, units = 'standard', startdate = '2017-01-01', enddate='2017-12-31', limit =365, offset = 25):
    url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=" +\
    datatypeid + "&stationid=" + stationid + "&units=" + units + "&startdate=" + startdate +\
    "&enddate=" + enddate + "&limit=" + str(limit)
    response = requests.get(url=url, headers=headers)
    station_weather_report_json = response.json()
    station_weather_report_df = pd.DataFrame(station_weather_report_json['results'])
    return(station_weather_report_df)

In [8]:
all_project_stations.head()

,datacoverage,elevation,elevationUnit,id,latitude,longitude,maxdate,mindate,name,square_diagonal,query_count,query_long,query_lat,dist_from_center,p_name
0,0.9995,304.8,METERS,GHCND_USW00013969,36.73667,-97.10194,2018-10-16,1948-01-01,"PONCA CITY MUNICIPAL AIRPORT, OK US",2.139864,14,-97.306297,36.572365,0.262218,Thunder Ranch
1,0.9872,299.9,METERS,GHCND_USW00003965,36.16240,-97.08940,2018-10-16,1996-11-01,"STILLWATER REGIONAL AIRPORT, OK US",2.139864,14,-97.306297,36.572365,0.463805,Thunder Ranch
2,0.9675,272.8,METERS,GHCND_USC00348501,36.11750,-97.09500,2018-10-17,1893-01-01,"STILLWATER 2 W, OK US",2.139864,14,-97.306297,36.572365,0.501546,Thunder Ranch
3,0.8607,1309.4,METERS,GHCND_USW00024030,41.09930,-102.98630,2018-10-16,1948-01-01,"SIDNEY MUNICIPAL AIRPORT, NE US",1.783220,13,-103.322228,40.961955,0.362920,Peetz Table
4,0.9565,1316.7,METERS,GHCND_USC00257830,41.22940,-103.02140,2018-09-30,1908-12-01,"SIDNEY 6 NNW, NE US",1.783220,13,-103.322228,40.961955,0.402522,Peetz Table


In [ ]:
station_count = 0
start_date = "2017-01-01"
end_date = "2017-12-31"
for index, row in all_project_stations.iterrows():
    stationid = row['id'].replace("_", ":")
    stationid_dir_name = row["id"]
    projectid = row["p_name"].replace("/","&")
    failed_datatypes = []
    station_count += 1
    for w in wind_data_types:
        try:
            print("Project:", projectid, " | Station:", stationid, " | Feature:", w, " | Station Count:", station_count)
            downloaded_data = get_wind_data_daily_summary(stationid, w, units = 'standard', startdate=start_date , enddate=end_date, limit=1000)
            file_path = './data/weather/projects/' + projectid + "/" + stationid_dir_name + '/' + w +\ 
            "_" + start_date + "_" + end_date + ".csv"
            downloaded_data.to_csv(file_path, index=False)
        except:
            failed_datatypes.append(w)
    with open('./data/weather/projects/' + projectid + "/" + stationid_dir_name + '/failed_datatypes' , 'w') as f:
        for item in failed_datatypes:
            f.write("%s\n" % item)

Project: Thunder Ranch  | Station: GHCND:USW00013969  | Feature: AWND  | Station Count: 1
Project: Thunder Ranch  | Station: GHCND:USW00013969  | Feature: DAWM  | Station Count: 1
Project: Thunder Ranch  | Station: GHCND:USW00013969  | Feature: FMTM  | Station Count: 1
Project: Thunder Ranch  | Station: GHCND:USW00013969  | Feature: HLY-WIND-1STDIR  | Station Count: 1
Project: Thunder Ranch  | Station: GHCND:USW00013969  | Feature: HLY-WIND-1STPCT  | Station Count: 1
Project: Thunder Ranch  | Station: GHCND:USW00013969  | Feature: HLY-WIND-2NDDIR  | Station Count: 1
Project: Thunder Ranch  | Station: GHCND:USW00013969  | Feature: HLY-WIND-2NDPCT  | Station Count: 1
Project: Thunder Ranch  | Station: GHCND:USW00013969  | Feature: HLY-WIND-AVGSPD  | Station Count: 1
Project: Thunder Ranch  | Station: GHCND:USW00013969  | Feature: HLY-WIND-PCTCLM  | Station Count: 1
Project: Thunder Ranch  | Station: GHCND:USW00013969  | Feature: HLY-WIND-VCTDIR  | Station Count: 1
Project: Thunder Ranch 

Project: Peetz Table  | Station: GHCND:USW00024030  | Feature: HLY-WIND-AVGSPD  | Station Count: 4
Project: Peetz Table  | Station: GHCND:USW00024030  | Feature: HLY-WIND-PCTCLM  | Station Count: 4
Project: Peetz Table  | Station: GHCND:USW00024030  | Feature: HLY-WIND-VCTDIR  | Station Count: 4
Project: Peetz Table  | Station: GHCND:USW00024030  | Feature: HLY-WIND-VCTSPD  | Station Count: 4
Project: Peetz Table  | Station: GHCND:USW00024030  | Feature: MDWM  | Station Count: 4
Project: Peetz Table  | Station: GHCND:USW00024030  | Feature: PGTM  | Station Count: 4
Project: Peetz Table  | Station: GHCND:USW00024030  | Feature: TWND  | Station Count: 4
Project: Peetz Table  | Station: GHCND:USW00024030  | Feature: WDF1  | Station Count: 4
Project: Peetz Table  | Station: GHCND:USW00024030  | Feature: WDF2  | Station Count: 4
Project: Peetz Table  | Station: GHCND:USW00024030  | Feature: WDF5  | Station Count: 4
Project: Peetz Table  | Station: GHCND:USW00024030  | Feature: WDFG  | Stati

Project: Horse Hollow  | Station: GHCND:USW00013962  | Feature: WDF5  | Station Count: 7
Project: Horse Hollow  | Station: GHCND:USW00013962  | Feature: WDFG  | Station Count: 7
Project: Horse Hollow  | Station: GHCND:USW00013962  | Feature: WDFI  | Station Count: 7
Project: Horse Hollow  | Station: GHCND:USW00013962  | Feature: WDFM  | Station Count: 7
Project: Horse Hollow  | Station: GHCND:USW00013962  | Feature: WDMV  | Station Count: 7
Project: Horse Hollow  | Station: GHCND:USW00013962  | Feature: WSF1  | Station Count: 7
Project: Horse Hollow  | Station: GHCND:USW00013962  | Feature: WSF2  | Station Count: 7
Project: Horse Hollow  | Station: GHCND:USW00013962  | Feature: WSF5  | Station Count: 7
Project: Horse Hollow  | Station: GHCND:USW00013962  | Feature: WSFG  | Station Count: 7
Project: Horse Hollow  | Station: GHCND:USW00013962  | Feature: WSFI  | Station Count: 7
Project: Horse Hollow  | Station: GHCND:USW00013962  | Feature: WSFM  | Station Count: 7
Project: Horse Hollow

Project: Brady Wind I  | Station: GHCND:USW00024012  | Feature: WSFG  | Station Count: 10
Project: Brady Wind I  | Station: GHCND:USW00024012  | Feature: WSFI  | Station Count: 10
Project: Brady Wind I  | Station: GHCND:USW00024012  | Feature: WSFM  | Station Count: 10
Project: Brady Wind I  | Station: GHCND:USC00322188  | Feature: AWND  | Station Count: 11
Project: Brady Wind I  | Station: GHCND:USC00322188  | Feature: DAWM  | Station Count: 11
Project: Brady Wind I  | Station: GHCND:USC00322188  | Feature: FMTM  | Station Count: 11
Project: Brady Wind I  | Station: GHCND:USC00322188  | Feature: HLY-WIND-1STDIR  | Station Count: 11
Project: Brady Wind I  | Station: GHCND:USC00322188  | Feature: HLY-WIND-1STPCT  | Station Count: 11
Project: Brady Wind I  | Station: GHCND:USC00322188  | Feature: HLY-WIND-2NDDIR  | Station Count: 11
Project: Brady Wind I  | Station: GHCND:USC00322188  | Feature: HLY-WIND-2NDPCT  | Station Count: 11
Project: Brady Wind I  | Station: GHCND:USC00322188  | F

Project: Flat Ridge 2  | Station: GHCND:USW00003957  | Feature: HLY-WIND-1STPCT  | Station Count: 14
Project: Flat Ridge 2  | Station: GHCND:USW00003957  | Feature: HLY-WIND-2NDDIR  | Station Count: 14
Project: Flat Ridge 2  | Station: GHCND:USW00003957  | Feature: HLY-WIND-2NDPCT  | Station Count: 14
Project: Flat Ridge 2  | Station: GHCND:USW00003957  | Feature: HLY-WIND-AVGSPD  | Station Count: 14
Project: Flat Ridge 2  | Station: GHCND:USW00003957  | Feature: HLY-WIND-PCTCLM  | Station Count: 14
Project: Flat Ridge 2  | Station: GHCND:USW00003957  | Feature: HLY-WIND-VCTDIR  | Station Count: 14
Project: Flat Ridge 2  | Station: GHCND:USW00003957  | Feature: HLY-WIND-VCTSPD  | Station Count: 14
Project: Flat Ridge 2  | Station: GHCND:USW00003957  | Feature: MDWM  | Station Count: 14
Project: Flat Ridge 2  | Station: GHCND:USW00003957  | Feature: PGTM  | Station Count: 14
Project: Flat Ridge 2  | Station: GHCND:USW00003957  | Feature: TWND  | Station Count: 14
Project: Flat Ridge 2  

...

Project: Flat Ridge 2 Expansion  | Station: GHCND:USW00003957  | Feature: HLY-WIND-PCTCLM  | Station Count: 26
Project: Flat Ridge 2 Expansion  | Station: GHCND:USW00003957  | Feature: HLY-WIND-VCTDIR  | Station Count: 26
Project: Flat Ridge 2 Expansion  | Station: GHCND:USW00003957  | Feature: HLY-WIND-VCTSPD  | Station Count: 26
Project: Flat Ridge 2 Expansion  | Station: GHCND:USW00003957  | Feature: MDWM  | Station Count: 26
Project: Flat Ridge 2 Expansion  | Station: GHCND:USW00003957  | Feature: PGTM  | Station Count: 26
Project: Flat Ridge 2 Expansion  | Station: GHCND:USW00003957  | Feature: TWND  | Station Count: 26
Project: Flat Ridge 2 Expansion  | Station: GHCND:USW00003957  | Feature: WDF1  | Station Count: 26
Project: Flat Ridge 2 Expansion  | Station: GHCND:USW00003957  | Feature: WDF2  | Station Count: 26
Project: Flat Ridge 2 Expansion  | Station: GHCND:USW00003957  | Feature: WDF5  | Station Count: 26
Project: Flat Ridge 2 Expansion  | Station: GHCND:USW00003957  | Fe

Project: Chisholm View  | Station: GHCND:USW00013969  | Feature: HLY-WIND-VCTSPD  | Station Count: 29
Project: Chisholm View  | Station: GHCND:USW00013969  | Feature: MDWM  | Station Count: 29
Project: Chisholm View  | Station: GHCND:USW00013969  | Feature: PGTM  | Station Count: 29
Project: Chisholm View  | Station: GHCND:USW00013969  | Feature: TWND  | Station Count: 29
Project: Chisholm View  | Station: GHCND:USW00013969  | Feature: WDF1  | Station Count: 29
Project: Chisholm View  | Station: GHCND:USW00013969  | Feature: WDF2  | Station Count: 29
Project: Chisholm View  | Station: GHCND:USW00013969  | Feature: WDF5  | Station Count: 29
Project: Chisholm View  | Station: GHCND:USW00013969  | Feature: WDFG  | Station Count: 29
Project: Chisholm View  | Station: GHCND:USW00013969  | Feature: WDFI  | Station Count: 29
Project: Chisholm View  | Station: GHCND:USW00013969  | Feature: WDFM  | Station Count: 29
Project: Chisholm View  | Station: GHCND:USW00013969  | Feature: WDMV  | Statio

Project: Kingman Wind Energy Center  | Station: GHCND:USW00013980  | Feature: MDWM  | Station Count: 32
Project: Kingman Wind Energy Center  | Station: GHCND:USW00013980  | Feature: PGTM  | Station Count: 32
Project: Kingman Wind Energy Center  | Station: GHCND:USW00013980  | Feature: TWND  | Station Count: 32
Project: Kingman Wind Energy Center  | Station: GHCND:USW00013980  | Feature: WDF1  | Station Count: 32
Project: Kingman Wind Energy Center  | Station: GHCND:USW00013980  | Feature: WDF2  | Station Count: 32
Project: Kingman Wind Energy Center  | Station: GHCND:USW00013980  | Feature: WDF5  | Station Count: 32
Project: Kingman Wind Energy Center  | Station: GHCND:USW00013980  | Feature: WDFG  | Station Count: 32
Project: Kingman Wind Energy Center  | Station: GHCND:USW00013980  | Feature: WDFI  | Station Count: 32
Project: Kingman Wind Energy Center  | Station: GHCND:USW00013980  | Feature: WDFM  | Station Count: 32
Project: Kingman Wind Energy Center  | Station: GHCND:USW0001398

Project: Capricorn Ridge  | Station: GHCND:USC00416499  | Feature: MDWM  | Station Count: 35
Project: Capricorn Ridge  | Station: GHCND:USC00416499  | Feature: PGTM  | Station Count: 35
Project: Capricorn Ridge  | Station: GHCND:USC00416499  | Feature: TWND  | Station Count: 35
Project: Capricorn Ridge  | Station: GHCND:USC00416499  | Feature: WDF1  | Station Count: 35
Project: Capricorn Ridge  | Station: GHCND:USC00416499  | Feature: WDF2  | Station Count: 35
Project: Capricorn Ridge  | Station: GHCND:USC00416499  | Feature: WDF5  | Station Count: 35
Project: Capricorn Ridge  | Station: GHCND:USC00416499  | Feature: WDFG  | Station Count: 35
Project: Capricorn Ridge  | Station: GHCND:USC00416499  | Feature: WDFI  | Station Count: 35
Project: Capricorn Ridge  | Station: GHCND:USC00416499  | Feature: WDFM  | Station Count: 35
Project: Capricorn Ridge  | Station: GHCND:USC00416499  | Feature: WDMV  | Station Count: 35
Project: Capricorn Ridge  | Station: GHCND:USC00416499  | Feature: WSF

Project: Brady Wind II  | Station: GHCND:USC00322188  | Feature: WDFG  | Station Count: 38
Project: Brady Wind II  | Station: GHCND:USC00322188  | Feature: WDFI  | Station Count: 38
Project: Brady Wind II  | Station: GHCND:USC00322188  | Feature: WDFM  | Station Count: 38
Project: Brady Wind II  | Station: GHCND:USC00322188  | Feature: WDMV  | Station Count: 38
Project: Brady Wind II  | Station: GHCND:USC00322188  | Feature: WSF1  | Station Count: 38
Project: Brady Wind II  | Station: GHCND:USC00322188  | Feature: WSF2  | Station Count: 38
Project: Brady Wind II  | Station: GHCND:USC00322188  | Feature: WSF5  | Station Count: 38
Project: Brady Wind II  | Station: GHCND:USC00322188  | Feature: WSFG  | Station Count: 38
Project: Brady Wind II  | Station: GHCND:USC00322188  | Feature: WSFI  | Station Count: 38
Project: Brady Wind II  | Station: GHCND:USC00322188  | Feature: WSFM  | Station Count: 38
Project: Brady Wind II  | Station: GHCND:USW00094038  | Feature: AWND  | Station Count: 39